# Tree species image classification using Artificial Neural Networks

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import os
import matplotlib.pyplot as plt

/home/sciapps/Documents/Repos/tfm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sciapps/Documents/Repos/tfm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sciapps/Documents/Repos/tfm/env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sciapps/Documents/Repos/tfm/env

## Getting the data

In [3]:
# DATA SET DIRECTORIES
source_dir = "tfm/data/images/image_preprocessing/processed_images/"
train_dir = os.path.join(source_dir, "train")
test_dir = os.path.join(source_dir, "test")
val_dir = os.path.join(source_dir, "val")
############################################
# LABELS
class_names = os.listdir(train_dir)
print(class_names)

FileNotFoundError: [Errno 2] No such file or directory: 'tfm/data/images/image_preprocessing/processed_images/train'

## Image decodification and augmentation

Aim: increase the number of examples by randomly applying transformations to the original images. It also prevents overfitting of the model. 

Augmnetation methods:

- Rotation.
- Width shift.
- Height shift.
- Horizontal flip
- Zooming

`ImageDataGenrator`:

- Read images from the disk.
- Decode images in arrays of float pixel values (here RGB).
- Rescale the floats in the arrays from values between 0 and 255 to 0 and 1.
- Perform real-time image augmentation.

`flow_from_directory`:

- Generate the batches of array image data (aka tensors) with the real-time data augmentation defined in the `ImageDataGenerator`.
- Resize the arrays.

In [ ]:
# CONFIGURATION
# Parameters for ImageDataGenerator 
color_mode= "rgb"   
img_height = 28 
img_width = 28   
class_mode="categorical" 
batch_size = 3 
epochs = 15                                     
shuffle=True                                                               
seed = 1234 
# Parameters for test dataset 
test_batch_size = 1

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                            rotation_range=45,
                                            width_shift_range=.15,
                                            height_shift_range=.15,
                                            horizontal_flip=True,
                                            zoom_range=0.5,
                                            validation_split = 0.16)  

In [ ]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                            target_size=(img_width, img_height),
                                            color_mode = color_mode,
                                            batch_size = batch_size,                               
                                            shuffle = shuffle,
                                            class_mode = class_mode,
                                            subset = "training",
                                            seed=seed
                                            ) 
validation_generator = train_datagen.flow_from_directory(train_dir,  # same directory as training data
                                                    target_size=(img_width, img_height),
                                                    color_mode = color_mode,
                                                    batch_size=batch_size,
                                                    class_mode= class_mode,
                                                    subset='validation',
                                                    seed=seed) # set as validation data 

## Creating, training and evaluating the model

### Model network architecture

The simplest network architecture constists of 3 layers:
- Input layer, with a number of nodes equal to the number of features in the model.
- Hidden layer, with a variable number of nodes. 
- Output layer, with a number of nodes equal to the number of classes. 

### Creating the model with `Keras`

In `Keras` the model is defined with the `Sequential` method as a linear stack ot layers.

Ty
the number of output channels for each Conv2D layer is controlled by the first argument (e.g., 32 or 64).
    ## Typically, as the width and height shrink, you can afford (computationally) to add more output channels in each Conv2D layer
    # For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
    # The tf.nn.softmax function converts these logits to "probabilities" for each class

In [ ]:
# MODEL PARAMETERS
array_size = img_width*img_height
dense_size = 128 
num_classes = 42
kernel_size = 3,3 # number of convolutional filters (width*height of the filter mask)
padding = "same" # case insensitive. Other option: "valid"

In [ ]:
# MODEL ARCHITECTURE
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(dense_size, activation = "relu", input_shape=(1,28,28,3)))
model.add(tf.keras.layers.Dense(num_classes, activation = "softmax")) # last layer
model.build()

In [ ]:
# COMPILING THE MODEL
# SparseCategoricalCrossentropi directly uses classes labels,
## so that they don't need to be numerically encoded.
optimizer = "adam" # Options: "sgd", "adam"
model.compile(optimizer=optimizer,
            loss = "sparse_categorical_crossentropy",
            #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
model.summary()

In [ ]:
# TRAINING THE MODEL
history = model.fit_generator(
    train_generator,
    steps_per_epoch=3, # batch_size,
    epochs=15,
    verbose=1, # get a progress bar and ETA
    validation_data=validation_generator,
    validation_steps=2 # batch_size
)